# 1. Data collection

## 1.1 Collect recall data from cpsc.gov

In [1]:
%%bash
    scrapy crawl CpscSpider -o downloads/cpsc.csv 2> downloads/cpsc_log.txt

## 1.2 Collect company data from compustat

In [1]:
print('placeholder')

placeholder


First, we setup our connection to the wrds database.

In [2]:
import wrds_tools
import parameters

wrds = wrds_tools.WrdsConnection(wrds_username=parameters.wrds_username)

/home/julian/PycharmProjects/corporate_disruptions/venv/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


Loading library list...
Done


Setup observation period and grab the basic info we need.

In [3]:
from datetime import date

wrds.set_observation_period(start_date=date(year=2009, month=1, day=1), 
                            end_date=date(year=2018, month=12, day=31))
wrds.build_sp500()

In [4]:
wrds.add_names()

wrds.add_industry_classifiers(get_GICS=True)

In [5]:
wrds.dataset.head()

,gvkey,name,SIC,NAICS,GICS_group,GICS_industry,GICS_sector,GICS_subindustry
0,001045,AMERICAN AIRLINES GROUP INC,4512,481111,2030,203020,20,20302010
1,001075,PINNACLE WEST CAPITAL CORP,4911,2211,5510,551010,55,55101010
2,001078,ABBOTT LABORATORIES,2834,325412,3510,351010,35,35101010
3,001161,ADVANCED MICRO DEVICES,3674,334413,4530,453010,45,45301020
4,001177,AETNA INC,6324,524114,3510,351020,35,35102030


### 1.2.1 Filter by industry

We use the GICS industry classification system to filter. For an overview, see https://en.wikipedia.org/wiki/Global_Industry_Classification_Standard . Industry Group 2550 is Retailing.

In [6]:
wrds.filter_by_industry(industry_code='2550', classification_system='GICS_group')

In [7]:
wrds.dataset['name'] = wrds.dataset['name'].apply(str.title)

print('Number of observations: ', len(wrds.dataset),
      '\n\n----------------\n')
print(wrds.dataset['name'])
wrds.dataset.head()

Number of observations:  42 

----------------

0                Best Buy Co Inc
1                    Target Corp
2            Dollar General Corp
3           Family Dollar Stores
4                     Macy'S Inc
5                        Gap Inc
6               Genuine Parts Co
7                 Home Depot Inc
8            Sears Holdings Corp
9                   L Brands Inc
10          Lowe'S Companies Inc
11                 Nordstrom Inc
12               Penney (J C) Co
13                Autonation Inc
14               Ross Stores Inc
15                Rs Legacy Corp
16               Foot Locker Inc
17             Tjx Companies Inc
18                  Big Lots Inc
19                  Tiffany & Co
20              Office Depot Inc
21           Signet Jewelers Ltd
22                   Staples Inc
23                  Autozone Inc
24                   Kohl'S Corp
25         Bed Bath & Beyond Inc
26       O'Reilly Automotive Inc
27                  Petsmart Inc
28          Urban Outfitters

,index,gvkey,name,SIC,NAICS,GICS_group,GICS_industry,GICS_sector,GICS_subindustry
0,41,002184,Best Buy Co Inc,5731,443142,2550,255040,25,25504020
1,83,003813,Target Corp,5331,452990,2550,255030,25,25503020
2,89,004016,Dollar General Corp,5331,452990,2550,255030,25,25503020
3,112,004560,Family Dollar Stores,5331,452990,2550,255030,25,25503020
4,115,004611,Macy'S Inc,5311,452111,2550,255030,25,25503010


### 1.2.2 Filter out Internet & Direct Marketing Retail

GICS Sub-Industry 25502020.

In [8]:
wrds.dataset[wrds.dataset['GICS_subindustry'] == '25502020']

,index,gvkey,name,SIC,NAICS,GICS_group,GICS_industry,GICS_sector,GICS_subindustry
33,562,064768,Amazon.Com Inc,5961,454111,2550,255020,25,25502020
34,594,119314,Booking Holdings Inc,7370,519130,2550,255020,25,25502020
35,607,126296,Expedia Group Inc,4700,561510,2550,255020,25,25502020
38,641,147579,Netflix Inc,7841,532230,2550,255020,25,25502020
41,711,199356,Tripadvisor Inc,7370,519130,2550,255020,25,25502020


In [9]:
wrds.dataset = wrds.dataset[wrds.dataset['GICS_subindustry'] != '25502020']
wrds.dataset = wrds.dataset.reset_index(drop=True)
wrds.dataset.head()

,index,gvkey,name,SIC,NAICS,GICS_group,GICS_industry,GICS_sector,GICS_subindustry
0,41,002184,Best Buy Co Inc,5731,443142,2550,255040,25,25504020
1,83,003813,Target Corp,5331,452990,2550,255030,25,25503020
2,89,004016,Dollar General Corp,5331,452990,2550,255030,25,25503020
3,112,004560,Family Dollar Stores,5331,452990,2550,255030,25,25503020
4,115,004611,Macy'S Inc,5311,452111,2550,255030,25,25503010


In [10]:
wrds.db.close()